# import 

In [ ]:
import tensorflow as tf 
import os 
from tensorflow.keras.layers import Dense , Conv2D, MaxPooling2D , Flatten , Input 
from tensorflow.keras.models import Model

In [ ]:
os.chdir('F:/facial-Verification')

# load the dataset 

In [ ]:
positive_images = tf.data.Dataset.list_files(os.path.join(os.getcwd() , 'data' , 'positive' , '*.jpg')).take(1000)
negative_images = tf.data.Dataset.list_files(os.path.join(os.getcwd() , 'data' , 'negative' , '*.jpg')).take(1000)
anchors_images = tf.data.Dataset.list_files(os.path.join(os.getcwd() , 'data' , 'anchors' , '*.jpg')).take(1000)

In [ ]:
def preprocess( anchor_path ,img_path  , label) : 
    # load anchor 
    anchor = tf.io.read_file(anchor_path ) 
    anchor_img = tf.io.decode_jpeg(anchor) 
    anchor_img = tf.image.resize(anchor_img  , (100 ,100))
    
    # load the image 
    img = tf.io.read_file(img_path) 
    img = tf.io.decode_jpeg(img) 
    img = tf.image.resize(img , (100 , 100))
     
    
    return anchor_img , img , label 

In [ ]:
positive_ds = tf.data.Dataset.zip(( anchors_images ,positive_images  , tf.data.Dataset.from_tensor_slices(tf.ones(len(positive_images)))))
negative_ds = tf.data.Dataset.zip(( anchors_images ,negative_images  , tf.data.Dataset.from_tensor_slices(tf.zeros(len(positive_images)))))

In [ ]:
ds = positive_ds.concatenate(negative_ds)

In [ ]:
ds = ds.map(preprocess).cache().shuffle(1000)  

In [ ]:
train_ds = ds.take(int(round(len(ds) * .75))) 
val_ds = ds.skip(int(round(len(ds) * .75)))

In [ ]:
train_ds = train_ds.batch(16).prefetch(8)
val_ds = val_ds.batch(16).prefetch(8)

# the model

In [ ]:
def make_embedding(): 
    inp = Input(shape=(100,100,3), name='input_image')
    
    # First block
    c1 = Conv2D(64, (10,10), activation='relu')(inp)
    m1 = MaxPooling2D(64, (2,2), padding='same')(c1)
    
    # Second block
    c2 = Conv2D(128, (7,7), activation='relu')(m1)
    m2 = MaxPooling2D(64, (2,2), padding='same')(c2)
    
    # Third block 
    c3 = Conv2D(128, (4,4), activation='relu')(m2)
    m3 = MaxPooling2D(64, (2,2), padding='same')(c3)
    
    # Final embedding block
    c4 = Conv2D(256, (4,4), activation='relu')(m3)
    f1 = Flatten()(c4)
    d1 = Dense(4096, activation='sigmoid')(f1)
    
    
    return Model(inputs=[inp], outputs=[d1], name='embedding')

In [ ]:
class L1Dist(tf.keras.layers.Layer):
    
    # Init method - inheritance
    def __init__(self, **kwargs):
        super().__init__()
       
    # Magic happens here - similarity calculation
    def call(self, input_embedding, validation_embedding):
        return tf.math.abs(input_embedding - validation_embedding)

In [ ]:
def make_siamese_model(): 
    
    # Anchor image input in the network
    input_image = Input(name='input_img', shape=(100,100,3))
    
    # Validation image in the network 
    validation_image = Input(name='validation_img', shape=(100,100,3))
    
    # Combine siamese distance components
    embedding = make_embedding()
    siamese_layer = L1Dist()
    siamese_layer._name = 'distance'
    distances = siamese_layer(embedding(input_image), embedding(validation_image))
    
    # Classification layer 
    classifier = Dense(1, activation='sigmoid')(distances)
    
    return Model(inputs=[input_image, validation_image], outputs=classifier, name='SiameseNetwork')

In [ ]:
siamese_model = make_siamese_model()

In [ ]:
siamese_model.summary()

In [ ]:
binary_cross_loss = tf.losses.BinaryCrossentropy()
opt = tf.keras.optimizers.Adam(1e-4)

# training loop

In [ ]:
@tf.function
def train_step(batch):
    
    # Record all of our operations 
    with tf.GradientTape() as tape:     
        # Get anchor and positive/negative image
        X = batch[:2]
        # Get label
        y = batch[2]
        
        # Forward pass
        yhat = siamese_model(X, training=True)
        # Calculate loss
        loss = binary_cross_loss(y, yhat)
    print(loss)
        
    # Calculate gradients
    grad = tape.gradient(loss, siamese_model.trainable_variables)
    
    # Calculate updated weights and apply to siamese model
    opt.apply_gradients(zip(grad, siamese_model.trainable_variables))
        
    # Return loss
    return loss

In [ ]:
from tensorflow.keras.metrics import Precision, Recall

In [ ]:
def train(data, EPOCHS):
    # Loop through epochs
    for epoch in range(1, EPOCHS+1):
        print('\n Epoch {}/{}'.format(epoch, EPOCHS))
        progbar = tf.keras.utils.Progbar(len(data))
        
        # Creating a metric object 
        r = Recall()
        p = Precision()
        
        # Loop through each batch
        for idx, batch in enumerate(data):
            # Run train step here
            loss = train_step(batch)
            yhat = siamese_model.predict(batch[:2])
            r.update_state(batch[2], yhat)
            p.update_state(batch[2], yhat) 
            progbar.update(idx+1)
        print(loss.numpy(), r.result().numpy(), p.result().numpy())
        
        

In [ ]:
train(train_ds, 5)

In [ ]:
siamese_model.save(os.path.join(os.getcwd() , 'model_1'))

In [ ]:
# Get a batch of test data
test_input, test_val, y_true = val_ds.as_numpy_iterator().next()

In [ ]:
y_hat = siamese_model.predict([test_input, test_val])